In [7]:
web2="https://physics.nist.gov/PhysRefData/XrayMassCoef/tab2.html"
web4="https://physics.nist.gov/PhysRefData/XrayMassCoef/tab4.html"

In [8]:
import pandas as pd
import numpy as np

In [21]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

req = Request(web4)
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
name=[]
for link in soup.findAll('a'):
    links.append('https://physics.nist.gov/PhysRefData/XrayMassCoef/'+link.get('href'))
    name.append(link.get('href'))

#print(links)

In [22]:
elementos=links[2:-6]
nombre=[X.replace('ComTab/',"")[:-5] for X in name[2:-6]]

In [53]:
SS=[]
mu=[]
muen=[]
for elem in elementos:
    url=elem
    A=pd.read_html(url, flavor = 'bs4')[1]
    if (A[A.eq('Energy').any(1)][2].values[0]=='μ/ρ'):
        A.dropna(subset = [0], inplace=True)
        B=A[[0,2,4]]
    else:
        A.dropna(subset = [2], inplace=True)
        B=A[[2,4,6]]
    B = B[4:]
    B.columns=['Energy','mu','muen']
    B=B.reset_index()
    B.drop('index',inplace=True,axis=1)
    B=B.set_index('Energy')
    SS.append(A[0][0].replace(' HTML table format',""))
    try:
        mu.append(B.astype(float).to_dict('series')['mu'])
        muen.append(B.astype(float).to_dict('series')['muen'])
    #mu.append(B.to_dict('series')['mu'])
    #muen.append(B.to_dict('series')['muen'])
    except:
        print(A[0][0].replace(' HTML table format',""))

C-552 Air-equivalent Plastic


In [61]:
SS.remove('C-552 Air-equivalent Plastic')

In [62]:
SS

['A-150 Tissue-Equivalent Plastic',
 'Glass, Lead',
 'Adipose Tissue (ICRU-44)',
 'Lithium Fluride',
 'Air, Dry (Near Sea Level)',
 'Lithium Tetraborate',
 'Alanine',
 'Lung Tissue (ICRU-44)',
 'Bakelite',
 'Magnesium Tetraborate',
 'Blood, Whole (ICRU-44)',
 'Mercuric Iodide',
 'Bone, Cortical (ICRU-44)',
 'Muscle, Skeletal (ICRU-44)',
 'B-100 Bone-Equivalent Plastic',
 'Ovary (ICRU-44)',
 'Brain, Grey/White Matter (ICRU-44)',
 'Photographic Emulsion (Kodak Type AA)',
 'Breast Tissue (ICRU-44)',
 'Photographic Emulsion (Standard Nuclear)',
 'Plastic Scintillator (Vinyltoluene)',
 'Cadmium Telluride',
 'Polyethylene',
 'Calcium Fluoride',
 'Polyethylene Terephthalate, "Mylar"',
 'Calcium Sulfate',
 'Polymethyl Methacrylate',
 '15\xa0mmol\xa0L-1 Ceric Ammonium Sulfate Solution',
 'Polystyrene',
 'Cesium Iodide',
 'Polytetrafluoroethylene, "Teflon"',
 'Concrete, Ordinary',
 'Polyvinyl Chloride',
 'Concrete, Barite (Type BA)',
 'Radiochromic Dye Film (Nylon Base)',
 'Eye Lens (ICRU-44)',


In [63]:
tabla1=pd.DataFrame(SS,columns=['Material'])

In [64]:
tabla1['mu']=mu

In [65]:
tabla1['muen']=muen

In [71]:
resultado=pd.merge(Material,tabla1,on='Material')

In [74]:
resultado.to_json('MassAttenuationElements.json',orient='index')

In [12]:
xcom_elements={}
for i,address in zip(Z,elementos):
    xcom_elements[i]=address

In [17]:
Material=pd.read_json('Compounds.json')

In [23]:
elementos[0]

'https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/a150.html'

In [38]:
A=pd.read_html(elementos[2], flavor = 'bs4')[1]

In [39]:
A[0][0].replace(' HTML table format',"")

'Adipose Tissue (ICRU-44)'

In [40]:
Material[Material['Material']==A[0][0].replace(' HTML table format',"")]

,Material,<Z/A>,I,Density,Composition
1,Adipose Tissue (ICRU-44),0.55579,64.8,0.95,"{'1': '0.114000', '6': '0.598000', '7': '0.007..."


In [57]:
Material=Material.drop('Unnamed: 0',axis=1)

In [58]:
Material.index = np.arange(1, len(Material) + 1)

In [59]:
Material

,Z,Element,Z/A,I,Density
1,H,Hydrogen,0.99212,19.2,0.000084
2,He,Helium,0.49968,41.8,0.000166
3,Li,Lithium,0.43221,40.0,0.534000
4,Be,Beryllium,0.44384,63.7,1.848000
5,B,Boron,0.46245,76.0,2.370000
...,...,...,...,...,...
88,Ra,Radium,0.38934,826.0,5.000000
89,Ac,Actinium,0.39202,841.0,10.070000
90,Th,Thorium,0.38787,847.0,11.720000
91,Pa,Protactinium,0.39388,878.0,15.370000


In [67]:
mu=[]
muen=[]
for i,row in Material.iterrows():
    url=xcom_elements[i]
    A=pd.read_html(url, flavor = 'bs4')[0]
    
    if (A[A.eq('Energy').any(1)][2].values[0]=='μ/ρ'):
        A.dropna(subset = [0], inplace=True)
        B=A[[0,2,4]]
    else:
        A.dropna(subset = [2], inplace=True)
        B=A[[2,4,6]]
    B = B[4:]
    B.columns=['Energy','mu','muen']
    B=B.reset_index()
    B.drop('index',inplace=True,axis=1)
    B=B.set_index('Energy')
    mu.append(B.astype(float).to_dict('series')['mu'])
    muen.append(B.astype(float).to_dict('series')['muen'])

In [69]:
Material['mu']=mu

In [70]:
Material['muen']=muen

In [71]:
Material

,Z,Element,Z/A,I,Density,mu,muen
1,H,Hydrogen,0.99212,19.2,0.000084,Energy 1.00000E-03 7.21700 1.50000E-03 2...,Energy 1.00000E-03 6.820000 1.50000E-03 ...
2,He,Helium,0.49968,41.8,0.000166,Energy 1.00000E-03 60.84000 1.50000E-03 ...,Energy 1.00000E-03 60.450000 1.50000E-03 ...
3,Li,Lithium,0.43221,40.0,0.534000,Energy 1.00000E-03 233.90000 1.50000E-03 ...,Energy 1.00000E-03 233.500000 1.50000E-03 ...
4,Be,Beryllium,0.44384,63.7,1.848000,Energy 1.00000E-03 604.10000 1.50000E-03 ...,Energy 1.00000E-03 603.500000 1.50000E-03 ...
5,B,Boron,0.46245,76.0,2.370000,Energy 1.00000E-03 1229.00000 1.50000E-03 ...,Energy 1.00000E-03 1228.00000 1.50000E-03 ...
...,...,...,...,...,...,...,...
88,Ra,Radium,0.38934,826.0,5.000000,Energy 1.00000E-03 6201.00000 1.02840E-03 ...,Energy 1.00000E-03 6187.00000 1.02840E-03 ...
89,Ac,Actinium,0.39202,841.0,10.070000,Energy 1.00000E-03 6469.00000 1.03923E-03 ...,Energy 1.00000E-03 6455.00000 1.03923E-03 ...
90,Th,Thorium,0.38787,847.0,11.720000,Energy 1.00000E-03 6614.00000 1.08088E-03 ...,Energy 1.00000E-03 6600.00000 1.08088E-03 ...
91,Pa,Protactinium,0.39388,878.0,15.370000,Energy 1.00000E-03 6530.00000 1.00334E-03 ...,Energy 1.00000E-03 6515.00000 1.00334E-03 ...


In [29]:
for z,mass in zip(nombre,elementos):
    url=mass
    A=pd.read_html(url, flavor = 'bs4')[0]
    
    if (A[A.eq('Energy').any(1)][2].values[0]=='μ/ρ'):
        A.dropna(subset = [0], inplace=True)
        B=A[[0,2,4]]
    else:
        A.dropna(subset = [2], inplace=True)
        B=A[[2,4,6]]
    B = B[4:]
    B.columns=['Energy','mu','muen']
    B=B.reset_index()
    B.drop('index',inplace=True,axis=1)
    B=B.set_index('Energy')
    masscoefi[z]=B.astype(float).to_dict('series')

NameError: name 'masscoefi' is not defined

In [30]:
url=xcom_elements[1]
A=pd.read_html(url, flavor = 'bs4')[0]
    
if (A[A.eq('Energy').any(1)][2].values[0]=='μ/ρ'):
    A.dropna(subset = [0], inplace=True)
    B=A[[0,2,4]]
else:
    A.dropna(subset = [2], inplace=True)
    B=A[[2,4,6]]
B = B[4:]
B.columns=['Energy','mu','muen']
B=B.reset_index()
B.drop('index',inplace=True,axis=1)
B=B.set_index('Energy')

In [75]:
Material.to_json('MassAttenuationCompounds.json',orient='index')

In [76]:
import json
with open('masscoefi.json', 'w') as fp:
    json.dump(Material.to_dict('index'), fp)

TypeError: Object of type Series is not JSON serializable